In [1]:
import pandas as pd
import numpy as np
from astropy.cosmology import Planck13
from astropy.io import fits
from astropy.table import Table
from tqdm import tqdm
%matplotlib inline
import matplotlib.pyplot as plt

import disperse

In [2]:
cosmo = Planck13
H0 = cosmo.H0.value
Om = cosmo.Om0
Ol = 0.69288
Ok = 0.0
print(f'H0 = {cosmo.H0.value}')
print(f'Om = {cosmo.Om0}')
print(f'Ol = {0.69288}')

H0 = 67.77
Om = 0.30712
Ol = 0.69288


In [3]:
import pickle

In [4]:
with open('srclist_lockman_032_wvsh.pkl', 'rb') as f:
    data = pickle.load(f)

In [5]:
data

,ID_SRC,RA,DEC,RADEC_ERR,EXT,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,EXT_LIKE,EXT_ERR,...,TSTOP,ML_EXP_1,APE_CTS_1,APE_BKG_1,APE_EXP_1,APE_EEF_1,APE_RADIUS_1,APE_POIS_1,RA_corr,DEC_corr
0,1,159.435510,57.198784,0.000000,0.000000,66750.085938,6.003547,2.523368,0.00000,0.000000,...,6.278688e+08,4871.123535,10428,14.270072,4870.865723,0.75,7.567884,0.000000,159.435370,57.198305
1,4494,164.608920,56.792861,0.000000,61.917984,1255.998169,2.845616,0.117522,403.74115,2.159959,...,6.278688e+08,781.437988,310,5.338651,782.237305,0.75,10.326495,0.000000,164.608780,56.792382
2,5,164.273234,58.077511,0.000000,0.000000,12358.708008,1.725359,0.764377,0.00000,0.000000,...,6.278688e+08,4826.105957,2642,13.689127,4825.153320,0.75,7.414415,0.000000,164.273094,58.077032
3,60,165.406673,57.491154,0.455702,0.000000,4928.606934,1.455636,0.044620,0.00000,0.000000,...,6.278688e+08,810.286499,909,3.674417,811.295105,0.75,10.214846,0.000000,165.406533,57.490675
4,4,166.406711,58.855927,0.246261,0.000000,14068.260742,1.441918,0.054748,0.00000,0.000000,...,6.278688e+08,2053.034180,2304,6.577723,2052.429688,0.75,7.917719,0.000000,166.406571,58.855448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8494,6212,155.836434,56.284402,3.276881,0.000000,8.493439,0.001906,0.000642,0.00000,0.000000,...,6.278688e+08,8058.040039,21,12.784801,8061.397949,0.75,5.642705,0.021427,155.836293,56.283923
8495,4451,160.883184,54.863436,3.520278,0.000000,7.994558,0.001858,0.000629,0.00000,0.000000,...,6.278688e+08,9023.207031,30,18.300381,9024.296875,0.75,6.462280,0.007378,160.883044,54.862957
8496,1694,160.930042,55.025946,4.273491,0.000000,6.765484,0.001818,0.000646,0.00000,0.000000,...,6.278688e+08,9252.168945,36,22.655256,9243.442383,0.75,7.039980,0.005829,160.929902,55.025467
8497,6914,161.200088,54.859166,3.911239,0.000000,6.506335,0.001811,0.000638,0.00000,0.000000,...,6.278688e+08,9293.346680,28,17.289949,9297.845703,0.75,6.206985,0.010835,161.199947,54.858687


In [6]:
data['RA'].min(), data['RA'].max()

(154.7626506540677, 167.09685093917133)

In [7]:
data['DEC'].min(), data['DEC'].max()

(54.267458818027215, 61.25285661021501)

In [8]:
data[data['EXT_LIKE'] > 0]

,ID_SRC,RA,DEC,RADEC_ERR,EXT,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,EXT_LIKE,EXT_ERR,...,TSTOP,ML_EXP_1,APE_CTS_1,APE_BKG_1,APE_EXP_1,APE_EEF_1,APE_RADIUS_1,APE_POIS_1,RA_corr,DEC_corr
1,4494,164.608920,56.792861,0.000000,61.917984,1255.998169,2.845616,0.117522,403.741150,2.159959,...,6.278688e+08,781.437988,310,5.338651,782.237305,0.75,10.326495,0.000000e+00,164.608780,56.792382
5,4236,164.542737,56.758416,0.000000,92.512520,201.090210,1.102708,0.102276,61.102184,5.957702,...,6.278688e+08,848.569397,64,5.489577,832.470581,0.75,10.218180,7.006492e-45,164.542597,56.757937
11,548,161.304824,58.144552,2.310791,42.419319,2784.219727,0.659592,0.019024,977.559204,1.294162,...,6.278688e+08,4887.219727,470,15.505969,4887.909180,0.75,7.571466,0.000000e+00,161.304683,58.144073
12,119,164.380062,57.993879,1.145211,22.034355,2786.380615,0.630641,0.015839,722.510925,0.826146,...,6.278688e+08,4679.162598,1081,13.731589,4678.761230,0.75,7.330606,0.000000e+00,164.379922,57.993400
13,29,160.251532,58.302621,0.000000,105.148270,223.150375,0.597084,0.049762,70.509407,5.488338,...,6.278688e+08,4862.271484,146,14.847513,4870.322754,0.75,7.559626,0.000000e+00,160.251392,58.302142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1539,7685,162.137056,59.906592,6.584505,15.248463,26.779800,0.017807,0.003101,10.934018,4.007318,...,6.278688e+08,4908.875488,50,14.756679,4907.821289,0.75,7.438211,5.065403e-13,162.136916,59.906113
1562,4623,155.656687,56.519299,6.153666,15.804686,52.165356,0.017601,0.002391,16.804056,3.008349,...,6.278688e+08,7132.506836,51,11.515516,7136.909180,0.75,5.785284,1.100906e-17,155.656547,56.518820
1646,7847,160.070787,59.723114,0.000000,20.348000,21.462021,0.016925,0.003189,9.474149,4.967316,...,6.278688e+08,4784.395996,38,12.474102,4785.064453,0.75,7.513441,4.759097e-09,160.070646,59.722635
1927,7717,159.332195,58.829966,5.118139,15.385753,24.035006,0.014844,0.002775,8.240784,4.087005,...,6.278688e+08,4874.981445,49,13.801598,4878.205566,0.75,7.570659,1.650315e-13,159.332055,58.829487


In [9]:
galaxies = pd.read_csv('SDSS/SDSS_DR16.csv')

In [10]:
galaxies

,ra,dec,z,zErr,class,subClass
0,146.71421,-1.041304,0.021275,0.000009,GALAXY,NaN
1,146.71421,-1.041304,0.021222,0.000009,GALAXY,STARFORMING
2,146.91945,-0.990492,0.213925,0.000052,GALAXY,NaN
3,146.74413,-0.652191,0.203783,0.000039,GALAXY,NaN
4,146.90229,-0.984913,0.652181,0.000133,QSO,BROADLINE
...,...,...,...,...,...,...
5789195,146.33588,26.600093,0.727322,0.000239,QSO,NaN
5789196,146.12242,26.516686,2.240312,0.001260,QSO,NaN
5789197,146.32252,26.549261,0.058894,0.000007,GALAXY,STARBURST
5789198,146.24366,26.573191,0.580577,0.000214,GALAXY,NaN


In [11]:
galaxies = pd.read_csv('SDSS/SDSS_DR16.csv')
galaxies = galaxies[galaxies['class'] == 'GALAXY']
galaxies = galaxies[(150 <= galaxies['ra']) & (galaxies['ra'] <= 172)]
galaxies = galaxies[(50 <= galaxies['dec']) & (galaxies['dec'] <= 65.5)]
galaxies = galaxies[(0.00 < galaxies['z']) & (galaxies['z'] <= 1.7)]
galaxies = galaxies[['ra', 'dec', 'z']]
galaxies.columns = ['RA', 'DEC', 'Z']
galaxies.drop_duplicates(subset=['RA', 'DEC', 'Z'])
galaxies.reset_index(drop=True, inplace=True)
galaxies

,RA,DEC,Z
0,150.92346,62.234634,0.198852
1,150.56499,62.336881,0.391256
2,151.25612,62.310918,0.044529
3,150.42803,62.094766,0.135773
4,151.15772,62.231892,0.135557
...,...,...,...
51851,150.10230,59.220185,0.861608
51852,150.34123,59.167977,0.350408
51853,150.31747,59.297759,0.448163
51854,150.37989,59.177747,0.589051


In [12]:
clusters = data.copy()
clusters = clusters[clusters['EXT_LIKE'] > 0]
clusters = clusters[['ID_SRC', 'RA', 'DEC']]
clusters.reset_index(drop=True, inplace=True)
clusters

,ID_SRC,RA,DEC
0,4494,164.608920,56.792861
1,4236,164.542737,56.758416
2,548,161.304824,58.144552
3,119,164.380062,57.993879
4,29,160.251532,58.302621
...,...,...,...
184,7685,162.137056,59.906592
185,4623,155.656687,56.519299
186,7847,160.070787,59.723114
187,7717,159.332195,58.829966


In [13]:
z = [round(0.05 + 0.002 * i, 5) for i in range(0, 751)]
# z = [round(0.05 + 0.05 * i, 5) for i in range(0, 31)]
# print(z)
# print('>>> ', z[-1])
ID, RA, DEC, Z = [], [], [], []
for i in range(clusters.shape[0]):
    row = clusters.iloc[i]
    ID = ID + [int(row['ID_SRC'])] * len(z)
    RA = RA + [row['RA']] * len(z)
    DEC = DEC + [row['DEC']] * len(z)
    Z = Z + z.copy()
len(ID), len(RA), len(DEC), len(Z)

(141939, 141939, 141939, 141939)

In [14]:
clusters = pd.DataFrame({'ID': ID, 'RA': RA, 'DEC': DEC, 'Z': Z})
clusters

,ID,RA,DEC,Z
0,4494,164.608920,56.792861,0.050
1,4494,164.608920,56.792861,0.052
2,4494,164.608920,56.792861,0.054
3,4494,164.608920,56.792861,0.056
4,4494,164.608920,56.792861,0.058
...,...,...,...,...
141934,7825,161.365965,54.637553,1.542
141935,7825,161.365965,54.637553,1.544
141936,7825,161.365965,54.637553,1.546
141937,7825,161.365965,54.637553,1.548


In [15]:
DPS = disperse.Disperse3D(
    galaxies, '_disperse_03/bin/',
    H0, Om, Ol, Ok,
    clusters=clusters
)

In [16]:
DPS.count_cart_coords()

In [17]:
SIGMA = 5.0
SMOOTH = 1
BOARD = 'smooth'
ANGLE = 30

In [18]:
sigmas = [
         0.2, 0.4, 0.6, 0.8,
    1.0, 1.2, 1.4, 1.6, 1.8,
    2.0, 2.2, 2.4, 2.6, 2.8,
    3.0, 3.2, 3.4, 3.6, 3.8, 
    4.0, 4.2, 4.4, 4.6, 4.8, 
    5.0, 5.2, 5.4, 5.6, 5.8, 
    6.0, 6.2, 6.4, 6.6, 6.8,
    7.0, 7.2, 7.4, 7.6, 7.8,
    8.0
]

In [19]:
# for sigma in sigmas:
#     SIGMA = sigma
#     print('### ', SIGMA)
#     DPS.apply_disperse(SIGMA, SMOOTH, BOARD, ANGLE)
#     DPS.save(f'lockman_dumps/{SIGMA}/')

In [20]:
cl_dists = []
for sigma in tqdm(sigmas):
    DPS = disperse.Disperse3D.read(f'lockman_dumps/{sigma}/')
    
    cl_conn, fil_conn, cl_min_dists \
        = DPS.count_conn([3] * DPS.clusters.shape[0])
    cl_dists.append(np.array(cl_min_dists)[None,:])
    
dists = np.concatenate(cl_dists, axis=0)

 18%|█▊        | 7/40 [08:55<42:03, 76.45s/it]   


KeyboardInterrupt: 

In [ ]:
dists = dists.T

In [ ]:
dists.shape

In [ ]:
rads = list(range(1, 11))

In [ ]:
true_sigmas = []

for i, rad in enumerate(rads):
    t = np.zeros(clusters.shape[0])
    for j, sigma in tqdm(enumerate(sigmas)):
#         DPS = disperse.Disperse3D.read(f'lockman_dumps/{sigma}/')

#         cl_conn, fil_conn, cl_dists \
#             = DPS.count_conn([rad] * DPS.clusters.shape[0])
#         cl_conn = np.array(cl_conn)
#         t[cl_conn > 0] = sigma
        t[dists[:,j] <= rad] = sigma

    true_sigmas.append(t)
    
true_sigmas = np.array(true_sigmas)

In [ ]:
true_sigmas = true_sigmas.T

In [ ]:
true_sigmas.shape

In [ ]:
true_sigmas_4 = true_sigmas[:,3]

In [ ]:
with open('coma_log_reg.pkl', 'rb') as f:
    log_reg = pickle.load(f)

In [ ]:
log_reg_scores = log_reg.predict_proba(dists)[:,1].reshape(-1)

In [ ]:
log_reg_scores

In [ ]:
np.corrcoef(true_sigmas[:,4], log_reg_scores)

In [ ]:
clusters

In [ ]:
t_01 = clusters.assign(sigma_4_score=true_sigmas[:,3])
t_01 = t_01.assign(log_reg_score=log_reg_scores)
t_01

In [ ]:
t_02 = data[['ID_SRC', 'EXT_LIKE']]
t_02.columns = ['ID', 'EXT_LIKE']
t_02

In [ ]:
res = pd.merge(t_01, t_02, left_on='ID', right_on='ID')
res = res.sort_values(by=['ID', 'Z'])
res.reset_index(drop=True, inplace=True)

In [ ]:
res

In [ ]:
res['ID'].unique()

In [ ]:
font = {'size': 16}
plt.rc('font', **font)
fig = plt.figure(figsize=(24, 12))
plt.grid()

ID = 8409
l = 300

plt.plot(
    res[res['ID'] == ID]['Z'][:l],
    res[res['ID'] == ID]['log_reg_score'][:l] * 8,
    label='log_reg'
)
plt.plot(
    res[res['ID'] == ID]['Z'][:l],
    res[res['ID'] == ID]['sigma_4_score'][:l],
    label='sigma',
    alpha=0.5,
    c='r'
)
plt.xlabel('Z')
plt.ylabel('score')
plt.legend()

In [ ]:
res[res['ID'] == ID]['EXT_LIKE']

In [ ]:
res[res['ID'] == 29]['log_reg_score'].plot()

In [ ]:
res[res['ID'] == 29]['sigma_4_score'].plot()

In [ ]:
DPS.clusters

In [ ]:
# x1 = -115.432782
# y1 = 31.776151
# z1 = 182.911628
# x2 = -228.104405
# y2 = 62.792215
# z2 = 361.447997
# ((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)**0.5